## 답변평가

In [ ]:
def evaluate_answer(state: InterviewState) -> InterviewState:
    ## 평가 진행 ##
    evaluation = state['evaluation'] # [{'':'', ...}, {}]
    llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.2, model_kwargs={"top_p":0.8})

    #parser = CommaSeparatedListOutputParser()

    s_msg="너는 여러 회사에서 면접 심사를 경험해본 경력 20년차 베테랑 면접관이야."
    h_msg="""
    면접자가 답변한 내용을 다음의 자료를 통해 상,중,하 중 하나로 평가를 진행해줘.
    나에게 답변 시, 답변예시처럼 답변해줘.
    - 질문내용:{current_question}
    - 답변내용:{user_answer}
    - 질문전략:{question_strategy}
    - 질문전략 부가설명:{question_strategy_direction}
    - 평가항목:질문과의 관련성, 답변의 구체성
    - 평가기준:
      - 평가항목: 질문과의 관련성
      - 해당항목 평가 기준:
        - 상 : 질문의 핵심 의도에 정확히 부합하며, 전반적인 내용을 명확히 다룸.
        - 중 : 질문과 관련은 있지만, 핵심 포인트가 부분적으로 누락됨.
        - 하 : 질문과 관련이 약하거나 엉뚱한 내용 중심.

      - 평가항목: 답변의 구체성
      - 해당항목 평가 기준:
        - 상 : 질문에 정확히 대응하며 핵심 내용이 일관성이 있음. 또한, 논리적이고 경험 중심의 구체적인 사례를 들어 설명함.
        - 중 : 경험을 언급하긴 하지만 세부 내용이 부족하거나 결과 중심이 아님.
        - 하 : 핵심 없이 두루뭉술하게 설명하거나, 질문의 요지를 제대로 파악하지 못함. 또한, 구체적인 정보가 없음.

      - 평가항목: 자기이해도
      - 해당항목 평가 기준:
        - 상 : 자신의 성격, 경험, 장단점을 명확히 이해하고 있으며, 그것이 선택에 어떻게 영향을 미쳤는지 일관되게 설명함.
        - 중 : 자기 경험과 특성을 인식하고 있으나, 깊이 있는 설명이나 구체적인 맥락이 부족함.
        - 하 : 자신의 특성이나 경험을 피상적으로 언급, 본인에 대한 이해가 부족함.

      - 평가항목: 두괄식 답변
      - 해당항목 평가 기준:
        - 상 : 첫 문장에서 요지를 명확히 전달, 이후 내용을 체계적으로 뒷반침한다. 듣는 사람이 핵심을 빠르게 이해할 수 있음.
        - 중 : 요지는 포함되어 있으나, 불분명하고 핵심이 중간이후에 드러남.
        - 하 : 핵심이 마지막에 가서야 언급이 됨. 전체적인 핵심이 무엇인지 파악하기 힘듬.

    - 답변예시
      - '저는 Python과 LangChain을 통해 AI agent를 구축해보았습니다.'
      - [중, 하, 중, 상]
      - 답변형식은 list이고 각 원소는 str형식이다.
      - 각 원소는 부가설명을 제외하고, <상/중/하> 중 하나이다.
      - list의 인덱스 0과 1번 자리는 각각 '질문과의 관련성', '답변의 구체성'을 의미한다.
      앞뒤로 ```python ~ ``` 붙이는것은 하지 마.
    """

    chat_promt = ChatPromptTemplate.from_messages([
              ("system",s_msg), ("human",h_msg),
    ])
    messages = chat_promt.format_messages(
        current_question=state["current_question"],
        user_answer=state["current_answer"],
        question_strategy=state["current_strategy"],
        question_strategy_direction=state["question_strategy"][state["current_strategy"]]['질문방향'],
        #format_instructions=parser.get_format_instructions()
    )

    response = llm(messages)
    #result = parser.parse(response.content)
    tmp = response.content
    result = list(tmp[1:-1].split(','))
    #return result

    evaluation.append({'질문과의 관련성':result[0].strip("' "), '답변의 구체성':result[1].strip("' "), '자기이해도':result[2].strip("' "), '두괄식 답변':result[3].strip("' ")})


    ## 질문 답변 내용 conversation에 추가 ##
    state["conversation"].append({'질문':state["current_question"], '답변':state["current_answer"], '질문전략':state["current_strategy"]})

    # return 코드는 제공합니다.
    return {
        **state,
        "evaluation": evaluation
    }


## Reflect

In [ ]:
def reflect(state: InterviewState) -> InterviewState:
    """
    평가 결과에 대한 자기 점검 노드
    """
    last_eval = state["evaluation"][-1]
    llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.2)

    s_msg = "너는 스스로 평가한 면접 결과를 점검하는 자아성찰 능력을 갖춘 AI 면접관이야."
    h_msg = """
    다음은 너(면접관)가 자동으로 평가한 결과야.
    이 평가가 논리적으로 타당하고 기준에 부합하는지 점검해줘.

    - 질문: {current_question}
    - 답변: {user_answer}
    - 질문전략: {question_strategy}
    - 질문전략 부가설명: {question_strategy_direction}
    - 평가결과:
        - 질문과의 관련성: {relevance}
        - 답변의 구체성: {specificity}
        - 자기이해도 : {self_aware}
        - 두괄식 답변: {answer}

    아래 형식으로 출력하세요:
    정상
    재평가

    앞뒤로 ```python ~ ``` 붙이는것은 하지 마.
    이유는 필요없어
    """

    chat_prompt = ChatPromptTemplate.from_messages([
        ("system", s_msg),
        ("human", h_msg),
    ])

    messages = chat_prompt.format_messages(
        current_question=state["current_question"],
        user_answer=state["current_answer"],
        question_strategy=state["current_strategy"],
        question_strategy_direction=state["question_strategy"][state["current_strategy"]]["질문방향"],
        relevance=last_eval["질문과의 관련성"],
        specificity=last_eval["답변의 구체성"],
        self_aware = last_eval["자기이해도"],
        answer = last_eval["두괄식 답변"],

    )

    response = llm(messages)

    return {
        **state,
        "next_step": response.content.strip()
    }

## 재평가

In [ ]:
def re_evaluate_answer(state: InterviewState) -> InterviewState:
    ## 평가 진행 ##
    evaluation = state['evaluation'] # [{'':'', ...}, {}]
    llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.2, model_kwargs={"top_p":0.8})

    #parser = CommaSeparatedListOutputParser()

    s_msg="너는 여러 회사에서 면접 심사를 경험해본 경력 20년차 베테랑 면접관이야."
    h_msg="""
    면접자가 답변한 내용을 다음의 자료를 통해 상,중,하 중 하나로 평가를 진행해줘.
    나에게 답변 시, 답변예시처럼 답변해줘.
    - 질문내용:{current_question}
    - 답변내용:{user_answer}
    - 질문전략:{question_strategy}
    - 질문전략 부가설명:{question_strategy_direction}
    - 평가항목:질문과의 관련성, 답변의 구체성
    - 평가기준:
      - 평가항목: 질문과의 관련성
      - 해당항목 평가 기준:
        - 상 : 질문의 핵심 의도에 정확히 부합하며, 전반적인 내용을 명확히 다룸.
        - 중 : 질문과 관련은 있지만, 핵심 포인트가 부분적으로 누락됨.
        - 하 : 질문과 관련이 약하거나 엉뚱한 내용 중심.

      - 평가항목: 답변의 구체성
      - 해당항목 평가 기준:
        - 상 : 질문에 정확히 대응하며 핵심 내용이 일관성이 있음. 또한, 논리적이고 경험 중심의 구체적인 사례를 들어 설명함.
        - 중 : 경험을 언급하긴 하지만 세부 내용이 부족하거나 결과 중심이 아님.
        - 하 : 핵심 없이 두루뭉술하게 설명하거나, 질문의 요지를 제대로 파악하지 못함. 또한, 구체적인 정보가 없음.

      - 평가항목: 자기이해도
      - 해당항목 평가 기준:
        - 상 : 자신의 성격, 경험, 장단점을 명확히 이해하고 있으며, 그것이 선택에 어떻게 영향을 미쳤는지 일관되게 설명함.
        - 중 : 자기 경험과 특성을 인식하고 있으나, 깊이 있는 설명이나 구체적인 맥락이 부족함.
        - 하 : 자신의 특성이나 경험을 피상적으로 언급, 본인에 대한 이해가 부족함.

      - 평가항목: 두괄식 답변
      - 해당항목 평가 기준:
        - 상 : 첫 문장에서 요지를 명확히 전달, 이후 내용을 체계적으로 뒷반침한다. 듣는 사람이 핵심을 빠르게 이해할 수 있음.
        - 중 : 요지는 포함되어 있으나, 불분명하고 핵심이 중간이후에 드러남.
        - 하 : 핵심이 마지막에 가서야 언급이 됨. 전체적인 핵심이 무엇인지 파악하기 힘듬.

    - 답변예시
      - '저는 Python과 LangChain을 통해 AI agent를 구축해보았습니다.'
      - [중, 하, 중, 상]
      - 답변형식은 list이고 각 원소는 str형식이다.
      - 각 원소는 부가설명을 제외하고, <상/중/하> 중 하나이다.
      - list의 인덱스 0과 1번 자리는 각각 '질문과의 관련성', '답변의 구체성'을 의미한다.
      앞뒤로 ```python ~ ``` 붙이는것은 하지 마.
    """

    chat_promt = ChatPromptTemplate.from_messages([
              ("system",s_msg), ("human",h_msg),
    ])
    messages = chat_promt.format_messages(
        current_question=state["current_question"],
        user_answer=state["current_answer"],
        question_strategy=state["current_strategy"],
        question_strategy_direction=state["question_strategy"][state["current_strategy"]]['질문방향'],
        #format_instructions=parser.get_format_instructions()
    )

    response = llm(messages)
    #result = parser.parse(response.content)
    tmp = response.content
    result = list(tmp[1:-1].split(','))
    #return result


    # 최신 평가 덮어쓰기
    evaluation[-1] ={'질문과의 관련성':result[0].strip("' "), '답변의 구체성':result[1].strip("' "), '자기이해도':result[2].strip("' "), '두괄식 답변':result[3].strip("' ")}


    # return 코드는 제공합니다.
    return {
        **state,
        "evaluation": evaluation,
        "next_step":""
    }


## 분기

In [ ]:
def route_reflect_next(state: InterviewState) -> Literal["re_evaluate", "decide"]:
    if state.get("reflect_result", "") == "재평가":
        return "re_evaluate"
    else:
        return "decide"